In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


from teste import diretorio_teste
project_path=diretorio_teste()

In [2]:
import os
import geopandas as gpd
import pandas as pd
from logging import (
    Logger,
    getLogger,
)
Logger=getLogger()

from notebooks.jupyter import utils
from utils import (
    save_parquet_excel,
    get_data_diretorio,
    check_crs
)

In [3]:
data_path = get_data_diretorio()

# Parques Urbanos e Lineares

# Dependências

Este notebook é dependente dos assets resultante dos notebooks '../../area_verde_publica/download_data/malha_parques_municipais.ipynb'

In [4]:
assets_path = os.path.join(
    data_path,
    'assets'
)

In [5]:
gdf_parq_mun = gpd.read_parquet(
    os.path.join(
        assets_path,
        'parques_municipais.parquet'
    )
)

# Conferir e Filtrar Parques Lineares e Urbanos

In [6]:
gdf_parq_mun.sample(2)

,cd_parq_mun,nm_parque,tx_tipo_ca,tx_tipo_si,cd_tipo_ca,cd_tipo_si,geometry
75,34,Sapé,Parque Linear,Existente,5,2,"POLYGON ((320729.853 7390900.271, 320739.122 7..."
201,49,Capadocia,Parque Urbano,Proposto,1,1,"MULTIPOLYGON (((322460.535 7385454.782, 322456..."


O gdf_parq_mun tem todos os parques em São Paulo. E há vários que não nos interessam (ps.: para garantir que não tenha problemas por causa da caixa alta ou letra capital, vamos deixar tudo em caixa baixa, em inglês é chamada de lower):

In [7]:
gdf_parq_mun['tx_tipo_ca']=gdf_parq_mun['tx_tipo_ca'].str.lower()

In [8]:
uniques=gdf_parq_mun['tx_tipo_ca'].unique()
#.str.contains('inear', case=False)
uniques

array(['parque urbano', 'parque orla', 'parque de conservação',
       'parque natural', 'parque linear'], dtype=object)

O mais seguro é filtrar pelo cd do tipo de parque. No nosso caso, a coluna [cd_tipo_ca]. Para termos certeza de qual o cd de cada um dos parques, vamos filtrar primeiro procurando similaridade nos textos dos tipos de parques, e então encontrar o cd correspondente.

In [9]:
for u in uniques:
    if 'linear' in str(u):
        name_linear = u
    if 'urb' in str(u):
        name_urb=u
print (f'{name_linear} | {name_urb}')

parque linear | parque urbano


In [10]:
cd_linear=(
    gdf_parq_mun
    .loc[gdf_parq_mun['tx_tipo_ca']==name_linear, 'cd_tipo_ca']
    .unique()
)

if int(cd_linear.shape[0])==1:
    cd_linear = int(cd_linear[0])
else:
    Logger.error(
        f'''Erro: o cd_tipo_ca ({
            cd_linear
        }) não corresponde completamente ao txt_tipo_ca({
            name_linear
        })'''
    )

print(f'O cd dos parques lineares é {cd_linear}')

O cd dos parques lineares é 5


In [11]:
cd_urb = (
    gdf_parq_mun
    .loc[gdf_parq_mun['tx_tipo_ca']==name_urb, 'cd_tipo_ca']
    .unique()
)

if int(cd_urb.shape[0])==1:
    cd_urb = int(cd_urb[0])
else:
    Logger.error(
        f'''Erro: o cd_tipo_ca ({
            cd_urb
        }) não corresponde completamente ao txt_tipo_ca({
            name_urb
        })'''
    )

print(f'O cd dos parques urbanos é {cd_urb}')

O cd dos parques urbanos é 1


Para unir os dois tipos de dados, vamos concatenar as linhas que correspondem a parques lineares com as linhas que correspondem a parques urbanos:

In [12]:
parques_line_urb=pd.concat([
    (
        gdf_parq_mun
        .loc[gdf_parq_mun['cd_tipo_ca']== cd_linear]
    ),
    (
        gdf_parq_mun
        .loc[gdf_parq_mun['cd_tipo_ca']==cd_urb]
    )
])

In [13]:
parques_line_urb.sample(3)

,cd_parq_mun,nm_parque,tx_tipo_ca,tx_tipo_si,cd_tipo_ca,cd_tipo_si,geometry
77,15,Aricanduva - Viaduto Badra,parque linear,Existente,5,2,"MULTIPOLYGON (((342242.038 7395972.418, 342241..."
96,17,Fonte do Peabiru,parque urbano,Proposto,1,1,"POLYGON ((324391.352 7392104.819, 324437.67 73..."
71,155,Nabuco,parque urbano,Existente,1,2,"POLYGON ((330686.739 7382244.04, 330694.701 73..."


In [14]:
dif = (gdf_parq_mun.shape[0])-(parques_line_urb.shape[0])
print(f'Há {dif} parques que não são nem urbanos, nem lineares')

Há 67 parques que não são nem urbanos, nem lineares


# Conferir CRS

In [15]:
parques_line_urb= check_crs(parques_line_urb)

# Salvar GDF

In [16]:
save_parquet_excel(
    parques_line_urb,
    fname='parques_urbanos_e_lineares',
    data_path = data_path,
    data_subpath = 'assets'
)